<a href="https://colab.research.google.com/github/capstoneaksara/capstone/blob/main/github_ML/HC_Aksara_dt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ipycanvas
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.1 MB/s eta 0:00:00


In [17]:
import tensorflow as tf
import numpy as np
import os
from tensorflow import keras
from google.colab import output
from ipycanvas import Canvas
from PIL import Image
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import Button, Output, VBox, HBox

In [4]:
# Move to the dataset directory
%cd /content/drive/My Drive/dataset

# Set the base path and verify that it's pointing to the correct directory
base_path = os.getcwd()  # Corrected to os.getcwd() to retrieve the current directory path
train_dir = os.path.join(base_path, 'train')
val_dir = os.path.join(base_path, 'val')
test_dir = os.path.join(base_path, 'test')

print("Base Path:", base_path)
print("Train Directory:", train_dir)
print("Validation Directory:", val_dir)
print("Test Directory:", test_dir)

/content/drive/My Drive/dataset
Base Path: /content/drive/My Drive/dataset
Train Directory: /content/drive/My Drive/dataset/train
Validation Directory: /content/drive/My Drive/dataset/val
Test Directory: /content/drive/My Drive/dataset/test


In [5]:
# Membuat model dengan CNN
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(150, 150, 3)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(50, activation='softmax')
    ])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 148, 148, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 15, 15, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 7, 7, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 3136)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 3136)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         803,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 50)                  │          12,850 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 876,434 (3.34 MB)

 Trainable params: 876,434 (3.34 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(150, 150),
    batch_size=128,
    label_mode='categorical'
    )

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(150, 150),
    batch_size=128,
    label_mode='categorical'
    )

# Optimize the datasets for training
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE

train_dataset_final = (train_dataset
                       .cache()
                       .shuffle(SHUFFLE_BUFFER_SIZE)
                       .prefetch(PREFETCH_BUFFER_SIZE)
                       )

validation_dataset_final = (validation_dataset
                            .cache()
                            .prefetch(PREFETCH_BUFFER_SIZE)
                            )

Found 8000 files belonging to 50 classes.
Found 2000 files belonging to 50 classes.


In [7]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
history = model.fit(
    train_dataset_final,
    epochs=20,
    validation_data = validation_dataset_final,
    verbose = 1
)

Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 1162s 4s/step - accuracy: 0.0360 - loss: 3.7212 - val_accuracy: 0.0525 - val_loss: 3.4496
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.0442 - loss: 3.4921 - val_accuracy: 0.0505 - val_loss: 3.4214
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.1148 - loss: 3.3080 - val_accuracy: 0.4660 - val_loss: 2.0232
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.4789 - loss: 1.9371 - val_accuracy: 0.5965 - val_loss: 1.4563
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.6263 - loss: 1.3581 - val_accuracy: 0.7070 - val_loss: 1.1296
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.7073 - loss: 1.0460 - val_accuracy: 0.7675 - val_loss: 0.9076
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.7651 - loss: 0.8230 - val_accuracy: 0.8040 - val_loss: 0.7731
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.8016 - loss: 0.7080 - val_accuracy: 0.8310 - 

In [9]:
# third party
output.enable_custom_widget_manager()

In [30]:
# Deklarasi canvas sebagai variabel global
global canvas
canvas = None

# Fungsi untuk membuat canvas dan menggambar
def create_canvas():
    global canvas
    canvas = Canvas(width=280, height=280, sync_image_data=True)
    canvas.fill_style = '#9E7F62'  # Background coklat lontar
    canvas.fill_rect(0, 0, 280, 280)  # Menyiapkan latar belakang coklat lontar
    canvas.stroke_style = 'black'  # Warna garis hitam
    canvas.line_width = 10  # Lebar garis
    drawing = False

    # Fungsi untuk menangani mouse
    def handle_mouse_down(x, y):
        nonlocal drawing
        drawing = True
        canvas.begin_path()
        canvas.move_to(x, y)

    def handle_mouse_move(x, y):
        if drawing:
            canvas.line_to(x, y)
            canvas.stroke()

    def handle_mouse_up(x, y):
        nonlocal drawing
        drawing = False

    canvas.on_mouse_down(handle_mouse_down)
    canvas.on_mouse_move(handle_mouse_move)
    canvas.on_mouse_up(handle_mouse_up)

    return canvas

# Proses gambar dari canvas untuk prediksi
def process_canvas_for_prediction(canvas):
    img_data = canvas.get_image_data(0, 0, canvas.width, canvas.height)
    img = np.array(img_data.data, dtype=np.uint8).reshape((canvas.height, canvas.width, 4))[:, :, 0]
    img = Image.fromarray(img)
    img = img.convert('L')
    img = img.resize((150, 150))
    img = np.array(img) / 255.0  # Normalisasi
    # Mengubah gambar menjadi 3 saluran (RGB)
    img = np.stack((img,)*3, axis=-1)  # Duplikasi saluran untuk membuat 3 saluran (RGB)
    img = img.reshape(1, 150, 150, 3)
    return img

# Fungsi untuk mengklasifikasikan gambar
def classify_drawing(model, canvas):
    img_array = process_canvas_for_prediction(canvas)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    print(f'Aksara: {predicted_class}')
    return predicted_class

# Tombol untuk klasifikasi
def run_classification(model):
    global canvas  # Pastikan canvas adalah variabel global
    canvas = create_canvas()  # Membuat canvas untuk menggambar
    display(canvas)  # Menampilkan canvas di Colab

# Tombol untuk klasifikasi
button = widgets.Button(description="Classify Aksara")
display(button)

def on_button_click(b):
    classify_drawing(model, canvas)  # Prediksi gambar yang digambar

button.on_click(on_button_click)

# Tombol untuk menghapus gambar
clear_button = widgets.Button(description="Clear")
display(clear_button)

def on_clear_click(b):
    canvas.clear()  # Bersihkan canvas untuk menggambar ulang
    canvas.fill_style = '#9E7F62'
    canvas.fill_rect(0, 0, 280, 280)

clear_button.on_click(on_clear_click)

# Jalankan fungsi utama untuk menerima input tulisan tangan
run_classification(model)


Button(description='Classify Aksara', style=ButtonStyle())

Button(description='Clear', style=ButtonStyle())

Canvas(height=280, sync_image_data=True, width=280)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Aksara: 0


In [ ]:
from google.colab import output
output.disable_custom_widget_manager()